In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob
from nltk import word_tokenize

mylist = list()
InputSheet = "Same-Sim-latestV1.csv"
MasterSheet = "Same_SIM_Master_List.csv"
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank','Comments'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv(MasterSheet)
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv(InputSheet,encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def GetRowNum(): 
    try:
        rownum = resultFrame.loc[(resultFrame.Accuracy == '0003T') & (resultFrame.Rank == 'CERVICOGRAPY')].index[0]
            #print(rownum)
        resultFrame.loc[rownum,'Comments'] = "Test"
    except:
        "No record found"

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,Comment,x):
    df = pd.read_csv(MasterSheet)    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    resultFrame.loc[x].Comments = Comment
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)   
    try:
        rowFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum])].index[0]  
        try:        
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            "Not Found"                                                                                   
    except:
        print(NewCodeSet[NewCodeNum])
        #print(i)
        test_set2 = word_tokenize(i)
        #tokenizer = TfidfVectorizer(stop_words = 'english')# initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
        tokenizer = TfidfVectorizer()
        score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
        #score
        pd.set_option("display.max_rows", 11)
        sortedscore = score.sort_values(by=['score'], ascending=False)
        sortedscore.head(11)
        ranknum = 0
        found = "false"
        for x in range(0,10):
            ranknum = ranknum+1
            df = pd.read_csv(MasterSheet) 
            svalue = sortedscore.iloc[x,1]
            GetCPTValue = df.iloc[svalue,0]
            if (GetCPTValue == SameSimSet[NewCodeNum]):
                found = "true"
                break 

        RecNum =0
        for x in range(0,10):
            rownum = rownum+1
            svalue = sortedscore.iloc[x,1]
            Accuracy = sortedscore.iloc[x,2]
            
            Comments = "Present in Autoproposal"

            if (RecNum ==0):
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,Comments,rownum)
            else:
                result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",Comments,rownum)
            RecNum =RecNum +1
            
        try:         
            rowNumSameSimFound = resultFrame.loc[(resultFrame.NewCode == NewCodeSet[NewCodeNum]) & (resultFrame.SameSimCode == SameSimSet[NewCodeNum])].index[0]
            resultFrame.loc[rowNumSameSimFound,'Comments'] = "Present Both in MD and Autoproposal"
        except:
            "Not Found"
resultFrame

export_csv = resultFrame.to_csv ('export_dataframe_V2.csv', index = None, header=True)



0045U
0046U
0047U
0048U
0049U
0050U
0051U
0052U
0053U
0054U
0055U
0056U
0057U
0058U
0059U
0060U
0061U
0062U
0063U
0064U
0065U
0066U
0067U
0068U
0069U
0070U
0071U
0072U
0073U
0074U
0075U
0076U
0077U
0078U
0079U
0505T
0506T
0507T
0508T
0509T 
0510T 
0511T
0512T
0513T
0514T
0515T
0516T
0517T
0518T
0519T
0520T
0521T
0522T
0523T
0524T
0525T
0526T
0527T
0528T
0529T
0530T
0531T
0532T
0533T
0534T
0535T
0536T
0537T
0538T
0539T
0540T
0541T
0542T
10004
10005
10006
10007
10008
10009
10010
10011
10012
11102
11103
11104
11105
11106
11107
20932
20933
20934
27369
33274
33275
33285
33286
33289
33440
33866
36572
36573
38531
43762
43763
50436
50437
53854
76391
76978
76979
76981
76982
76983
77046
77047
77048
77049
81163
81164
81165
81166
81167
81171
81172
81173
81174
81177
81178
81179
81180
81181
81182
81183
81184
81185
81186
81187
81188
81189
81190
81204
81233
81234
81236
81237
81239
81271
81274
81284
81285
81286
81289
81305
81306
81312
81320
81329
81333
81336
81337
81343
81344
81345
81443
81518
81596
82